In [2]:
import sys
import os

sys.path.append(os.path.abspath(".."))


from sortabilitytime.sortnregress_time import (
    sortnregress_ts,
    r2_sortnregress_ts,
    random_sortnregress_ts,
    sortnregress_ts_reverse,
    __ts_graph_to_summary_graph as ts_graph_to_summary_graph,
)
from sortabilitytime.dynotears import Dynotears

from sortabilitytime.sortability_ts import var_sortability, r2_sortability

# from causalnex.structure import dynotears
from tigramite.pcmci import PCMCI
from tigramite.independence_tests.parcorr import ParCorr
import tigramite.data_processing as pp

import tigramite.data_processing as pp
from typing import List, Tuple
import numpy as np
import scipy.linalg as slin
from scipy.ndimage import shift
import scipy.optimize as sopt
import warnings

from tigramite import plotting as tp
import matplotlib.pyplot as plt
import networkx as nx

from dynotears import Dynotears

from sklearn.metrics import auc, roc_curve, f1_score
from tqdm import tqdm

# import standardscaler
from sklearn.preprocessing import StandardScaler
import tigramite.data_processing as pp


In [3]:
import pandas as pd

In [4]:
from causalchamber.datasets import Dataset

# Download the dataset and store it, e.g., in the current directory
dataset = Dataset(name="lt_malus_v1", root="../../Data", download=True)
print(dataset.available_experiments())
# Select an experiment and load its observations
experiment = dataset.get_experiment(name="green_64")
df = experiment.as_pandas_dataframe()


Fetching list of available datasets from https://causalchamber.s3.eu-central-1.amazonaws.com/downloadables/directory.yaml ... done.
If you use our datasets or simulators for your work please consider citing:

﻿@article{gamella2025chamber,
  author={Gamella, Juan L. and Peters, Jonas and B{"u}hlmann, Peter},
  title={Causal chambers as a real-world physical testbed for {AI} methodology},
  journal={Nature Machine Intelligence},
  doi={10.1038/s42256-024-00964-x},
  year={2025},
}

Dataset lt_malus_v1 found in "../../Data/lt_malus_v1".
['red_255', 'white_128', 'green_64', 'blue_255', 'green_128', 'white_64', 'blue_64', 'green_255', 'blue_128', 'white_255', 'red_64', 'red_128']


In [5]:
df.columns

Index(['timestamp', 'config', 'counter', 'flag', 'intervention', 'red',
       'green', 'blue', 'osr_c', 'v_c', 'current', 'pol_1', 'pol_2',
       'osr_angle_1', 'osr_angle_2', 'v_angle_1', 'v_angle_2', 'angle_1',
       'angle_2', 'ir_1', 'vis_1', 'ir_2', 'vis_2', 'ir_3', 'vis_3', 'l_11',
       'l_12', 'l_21', 'l_22', 'l_31', 'l_32', 'diode_ir_1', 'diode_vis_1',
       'diode_ir_2', 'diode_vis_2', 'diode_ir_3', 'diode_vis_3', 't_ir_1',
       't_vis_1', 't_ir_2', 't_vis_2', 't_ir_3', 't_vis_3', 'camera',
       'v_board', 'v_reg'],
      dtype='object')

In [6]:
from causalchamber.ground_truth import graph

ground_truth = graph(chamber="lt", configuration="standard")

# Output:

#              red  green  blue  osr_c  v_c  current  pol_1  pol_2  osr_angle_1  \
# red            0      0     0      0    0        1      0      0            0
# green          0      0     0      0    0        1      0      0            0
# blue           0      0     0      0    0        1      0      0            0
# osr_c          0      0     0      0    0        1      0      0            0

In [7]:
matching_columns = list( set(ground_truth.columns) & set(df.columns))

matching_columns

['l_11',
 't_vis_1',
 'angle_2',
 't_ir_2',
 'angle_1',
 'vis_2',
 'diode_vis_2',
 't_ir_3',
 'vis_3',
 'ir_2',
 'diode_ir_3',
 'pol_1',
 't_vis_2',
 'v_c',
 'v_angle_2',
 'diode_ir_1',
 'diode_ir_2',
 'osr_angle_2',
 'green',
 'red',
 'l_21',
 'l_22',
 'diode_vis_3',
 'blue',
 'osr_c',
 'current',
 'l_31',
 't_vis_3',
 'l_12',
 't_ir_1',
 'pol_2',
 'ir_3',
 'l_32',
 'v_angle_1',
 'vis_1',
 'osr_angle_1',
 'ir_1',
 'diode_vis_1']

In [8]:
df = df[matching_columns]

df.columns

Index(['l_11', 't_vis_1', 'angle_2', 't_ir_2', 'angle_1', 'vis_2',
       'diode_vis_2', 't_ir_3', 'vis_3', 'ir_2', 'diode_ir_3', 'pol_1',
       't_vis_2', 'v_c', 'v_angle_2', 'diode_ir_1', 'diode_ir_2',
       'osr_angle_2', 'green', 'red', 'l_21', 'l_22', 'diode_vis_3', 'blue',
       'osr_c', 'current', 'l_31', 't_vis_3', 'l_12', 't_ir_1', 'pol_2',
       'ir_3', 'l_32', 'v_angle_1', 'vis_1', 'osr_angle_1', 'ir_1',
       'diode_vis_1'],
      dtype='object')

In [9]:
ground_truth = ground_truth[matching_columns]

ground_truth

# set index to order of matching columns remove if it does not appear

ground_truth = ground_truth.reindex(index=matching_columns)

In [10]:
ground_truth

l_11  t_vis_1  angle_2  t_ir_2  angle_1  vis_2  diode_vis_2  \
l_11            0        0        0       0        0      0            0   
t_vis_1         0        0        0       0        0      0            0   
angle_2         0        0        0       0        0      0            0   
t_ir_2          0        0        0       0        0      0            0   
angle_1         0        0        0       0        0      0            0   
vis_2           0        0        0       0        0      0            0   
diode_vis_2     0        0        0       0        0      1            0   
t_ir_3          0        0        0       0        0      0            0   
vis_3           0        0        0       0        0      0            0   
ir_2            0        0        0       0        0      0            0   
diode_ir_3      0        0        0       0        0      0            0   
pol_1           0        0        0       0        1      0            0   
t_vis_2         0        0        0       0        0      1            0   
v_c             0        0        0       0        0      0            0   
v_angle_2       0        0        1       0        0      0            0   
diode_ir_1      0        0        0       0        0      0            0   
diode_ir_2      0        0        0       0        0      0            0   
osr_angle_2     0        0        1       0        0      0            0   
green           0        0        0       0        0      1            0   
red             0        0        0       0        0      1            0   
l_21            0        0        0       0        0      1            0   
l_22            0        0        0       0        0      1            0   
diode_vis_3     0        0        0       0        0      0            0   
blue            0        0        0       0        0      1            0   
osr_c           0        0        0       0        0      0            0   
current         0        0        0       0        0      0            0   
l_31            0        0        0       0        0      0            0   
t_vis_3         0        0        0       0        0      0            0   
l_12            0        0        0       0        0      0            0   
t_ir_1          0        0        0       0        0      0            0   
pol_2           0        0        1       0        0      0            0   
ir_3            0        0        0       0        0      0            0   
l_32            0        0        0       0        0      0            0   
v_angle_1       0        0        0       0        1      0            0   
vis_1           0        0        0       0        0      0            0   
osr_angle_1     0        0        0       0        1      0            0   
ir_1            0        0        0       0        0      0            0   
diode_vis_1     0        0        0       0        0      0            0   

             t_ir_3  vis_3  ir_2  ...  l_12  t_ir_1  pol_2  ir_3  l_32  \
l_11              0      0     0  ...     0       0      0     0     0   
t_vis_1           0      0     0  ...     0       0      0     0     0   
angle_2           0      0     0  ...     0       0      0     0     0   
t_ir_2            0      0     1  ...     0       0      0     0     0   
angle_1           0      0     0  ...     0       0      0     0     0   
vis_2             0      0     0  ...     0       0      0     0     0   
diode_vis_2       0      0     0  ...     0       0      0     0     0   
t_ir_3            0      0     0  ...     0       0      0     1     0   
vis_3             0      0     0  ...     0       0      0     0     0   
ir_2              0      0     0  ...     0       0      0     0     0   
diode_ir_3        0      0     0  ...     0       0      0     1     0   
pol_1             0      1     0  ...     0       0      0     1     0   
t_vis_2           0      0     0  ...     0       0      0     0     0   
v_c         

In [11]:
df = df[ground_truth.columns]

In [12]:
ground_truth.to_numpy()

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(38, 38))

In [13]:
# renove nan and infs from df
import numpy as np
df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna()

In [14]:
ground_truth = ground_truth.replace([np.inf, -np.inf], np.nan)

In [15]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [16]:
from tigramite.independence_tests.parcorr import ParCorr

import numpy as np

parcorr = ParCorr(
    significance="analytic",
    #                   mask_type='y'
)

In [17]:
tau_max = 2

In [ ]:
# Provided dataset information
dataset_info = [
    {"name": "lt_camera_walks_v1", "chamber": "Light tunnel", "config": "camera"},
    {"name": "lt_color_regression_v1", "chamber": "Light tunnel", "config": "camera"},
    {
        "name": "lt_interventions_standard_v1",
        "chamber": "Light tunnel",
        "config": "standard",
    },
    {"name": "lt_walks_v1", "chamber": "Light tunnel", "config": "standard"},
    {"name": "wt_walks_v1", "chamber": "Wind tunnel", "config": "standard"},
    {"name": "lt_malus_v1", "chamber": "Light tunnel", "config": "standard"},
    {"name": "wt_bernoulli_v1", "chamber": "Wind tunnel", "config": "standard"},
    {"name": "wt_changepoints_v1", "chamber": "Wind tunnel", "config": "standard"},
    {"name": "wt_intake_impulse_v1", "chamber": "Wind tunnel", "config": "standard"},
    {
        "name": "wt_pressure_control_v1",
        "chamber": "Wind tunnel",
        "config": "pressure-control",
    },
    {"name": "lt_test_v1", "chamber": "Light tunnel", "config": "standard"},
    {"name": "wt_test_v1", "chamber": "Wind tunnel", "config": "standard"},
    {"name": "lt_camera_test_v1", "chamber": "Light tunnel", "config": "camera"},
    {"name": "wt_validate_v1", "chamber": "Wind tunnel", "config": "standard"},
    {
        "name": "wt_pc_validate_v1",
        "chamber": "Wind tunnel",
        "config": "pressure-control",
    },
    {"name": "lt_validate_v1", "chamber": "Light tunnel", "config": "standard"},
    {"name": "lt_camera_validate_v1", "chamber": "Light tunnel", "config": "standard"},
]

# Initialize an empty dictionary to store results
results_dict = {}

f1_scores = {
    "f1_var_sortnregress": [],
    "f1_r2_sortnregress": [],
    "f1_random": [],
}

# Loop through each dataset
for dataset_info_entry in dataset_info:
    dataset_name = dataset_info_entry["name"]
    chamber = dataset_info_entry["chamber"]
    config = dataset_info_entry["config"]

    # Load dataset based on chamber
    if chamber.lower() == "wind tunnel":
        short_name = "wt"
    elif chamber.lower() == "light tunnel":
        short_name = "lt"

    # Assuming the dataset is fetched and processed here
    # df = fetch_and_process_dataset(dataset_name)

    # Placeholder dataframe
    dataset = Dataset(name=dataset_name, root="../../Data", download=True)
    ground_truth = graph(chamber=short_name, configuration=config)

    for experiment in dataset.available_experiments():
        ground_truth = graph(chamber=short_name, configuration=config)

        # combine above print statements

        print(
            f"running {experiment} on {dataset_name} with config {config} and chamber {short_name}"
        )
        df = dataset.get_experiment(name=experiment).as_pandas_dataframe()

        # remove nans and infs from df
        df = df.replace([np.inf, -np.inf], np.nan)
        df = df.dropna()

        try:
            df = df[ground_truth.columns]

        except KeyError as e:
            print(
            f"error in {experiment} on {dataset_name} with config {config} and chamber {short_name}"
            )
            # remove [im] column from ground_truth
            ground_truth = ground_truth.drop(columns=["im"])
            # remove [im] from index
            ground_truth = ground_truth.drop(index=["im"])
            df = df[ground_truth.columns]
        # update results_dict with var and r2 sortability

        var = var_sortability(df.to_numpy(), ground_truth.to_numpy())
        r2 = r2_sortability(df.to_numpy(), ground_truth.to_numpy())

        # remove infs and nans in gruond_truth
        ground_truth = ground_truth.replace([np.inf, -np.inf], np.nan)
        ground_truth = ground_truth.dropna()

        # also in data
        df = df.replace([np.inf, -np.inf], np.nan)

        df = df.dropna()

        data = df.to_numpy()

        dataframe = pp.DataFrame(
            data.copy(), mask=None, datatime={0: np.arange(len(data))}
        )
        # dynotears = Dynotears(dataframe=dataframe)
        # w_est, a_est, string_graph, a_est_dyno = dynotears.run_dynotears(
        #     tau_max=3, max_iter=100, w_threshold=0.1, lambda_a=0.1, lambda_w=0.1
        # )

        # a_est_dyno = ts_graph_to_summary_graph(a_est_dyno)

        # f1_dynotears = f1_score(ground_truth.to_numpy().flatten(), a_est_dyno.flatten())
        try:
            data = df.to_numpy()
            dataframe = pp.DataFrame(data)
            pcmci = PCMCI(dataframe=dataframe, cond_ind_test=parcorr, verbosity=0)
            results_pcmci = pcmci.run_pcmciplus(tau_max=3, pc_alpha=0.01)
            results_pcmci["p_matrix"] = np.where(
                results_pcmci["p_matrix"] < 0.01, results_pcmci["p_matrix"], 0
            )
            a_pcmci = results_pcmci["p_matrix"]
            a_pcmci[a_pcmci != 0] = 1

            a_pcmci = ts_graph_to_summary_graph(a_pcmci)

            G_true = ground_truth.to_numpy()

            a_est_var_sortnregress = sortnregress_ts(data, tau_max=tau_max)

            a_est_var_sortnregress_reverse = sortnregress_ts_reverse(data, tau_max=tau_max)
            a_est_r2_sortnregress = r2_sortnregress_ts(data, tau_max=tau_max)
            a_est_random = random_sortnregress_ts(data, tau_max=tau_max)

            a_est_var_sortnregress = ts_graph_to_summary_graph(a_est_var_sortnregress)

            a_est_var_sortnregress_reverse = ts_graph_to_summary_graph(
                a_est_var_sortnregress_reverse
            )

            a_est_r2_sortnregress = ts_graph_to_summary_graph(a_est_r2_sortnregress)

            a_est_random = ts_graph_to_summary_graph(a_est_random)

            # replace all values above !=0 with 1
            a_est_var_sortnregress[a_est_var_sortnregress != 0] = 1
            a_est_r2_sortnregress[a_est_r2_sortnregress != 0] = 1
            a_est_random[a_est_random != 0] = 1
            f1_var_sortnregress = f1_score(
                G_true.flatten(), a_est_var_sortnregress.flatten()
            )

            f1_var_sortnregress_reverse = f1_score(
                G_true.flatten(), a_est_var_sortnregress_reverse.flatten()
            )
            f1_r2_sortnregress = f1_score(G_true.flatten(), a_est_r2_sortnregress.flatten())
            f1_random = f1_score(G_true.flatten(), a_est_random.flatten())
            f1_pcmci = f1_score(G_true.flatten(), a_pcmci.flatten())

            result = {
                "var_sortability": var,
                "r2_sortability": r2,
                # "f1_dynotears": f1_dynotears,
                "f1_pcmci": f1_pcmci,
                "f1_var_sortnregress": f1_var_sortnregress,
                "f1_var_sortnregress_reverse": f1_var_sortnregress_reverse,
                "f1_r2_sortnregress": f1_r2_sortnregress,
                "f1_random": f1_random,
            }
            results_dict[f"{dataset_name}+{experiment}"] = result

        except:
            result = {
                "var_sortability": var,
                "r2_sortability": r2,
                # "f1_dynotears": f1_dynotears,
                "f1_pcmci": np.nan,
                "f1_var_sortnregress": np.nan,
                "f1_var_sortnregress_reverse": np.nan,
                "f1_r2_sortnregress": np.nan,
                "f1_random": np.nan,
            }
            results_dict[f"{dataset_name}+{experiment}"] = result

Dataset lt_camera_walks_v1 found in "../../Data/lt_camera_walks_v1".
running color_mix on lt_camera_walks_v1 with config camera and chamber lt
error in color_mix on lt_camera_walks_v1 with config camera and chamber lt
fallback


In [ ]:
# # save results_dict to json
# import json

# with open("results_causalchamber.json", "w") as f:
#     json.dump(results_dict, f)

In [21]:
df = pd.DataFrame(results_dict)

df = df.T

# remove every value with no + in index
for index in df.index:
    if "+" not in index:
        df = df.drop(index)


# round to 2 decimals
df = df.round(2)

# experiment column is after the + in index
df["Experiment"] = df.index.str.split("+").str[1]

# dataset column is before the + in index
df["Dataset"] = df.index.str.split("+").str[0]

# remove index and set dataset as first and experiment as second column
df = df.reset_index(drop=True)

# df = df[["Dataset", "Experiment", "var_sortability", "r2_sortability"]]

# df.to_latex("results_causalchamber.txt")



In [23]:
# Define the columns to aggregate
agg_columns = [
    "var_sortability",
    "r2_sortability",
    "f1_pcmci",
    "f1_var_sortnregress",
    "f1_var_sortnregress_reverse",
    "f1_r2_sortnregress",
    "f1_random",
]

# Group by Dataset and compute mean and std for each column
df_grouped = df.groupby("Dataset").agg({col: ["mean", "std"] for col in agg_columns})

# Round to 2 decimal places
df_grouped = df_grouped.round(2)

# Replace NaNs with 0
df_grouped = df_grouped.fillna(0)

# Flatten multiindex columns
df_grouped.columns = ["_".join(col) for col in df_grouped.columns]

# Combine mean and std into "mean ± std" strings for each metric
for col in agg_columns:
    mean_col = f"{col}_mean"
    std_col = f"{col}_std"
    df_grouped[col] = (
        df_grouped[mean_col].astype(str) + " $\pm$ " + df_grouped[std_col].astype(str)
    )

# Drop the separate mean and std columns
df_grouped = df_grouped.drop(
    columns=[f"{col}_mean" for col in agg_columns]
    + [f"{col}_std" for col in agg_columns]
)

# Optional: reset index if you want a clean table
# df_grouped = df_grouped.reset_index()

# Show the final result
df_grouped

var_sortability   r2_sortability         f1_pcmci  \
Dataset                                                                    
lt_camera_walks_v1      0.95 $\pm$ 0.0  0.23 $\pm$ 0.07  0.25 $\pm$ 0.04   
lt_color_regression_v1  0.89 $\pm$ 0.0   0.21 $\pm$ 0.0   0.18 $\pm$ 0.0   

                       f1_var_sortnregress f1_var_sortnregress_reverse  \
Dataset                                                                  
lt_camera_walks_v1         0.36 $\pm$ 0.02             0.06 $\pm$ 0.01   
lt_color_regression_v1       0.1 $\pm$ 0.0               0.0 $\pm$ 0.0   

                       f1_r2_sortnregress       f1_random  
Dataset                                                    
lt_camera_walks_v1        0.26 $\pm$ 0.11  0.2 $\pm$ 0.02  
lt_color_regression_v1      0.0 $\pm$ 0.0   0.0 $\pm$ 0.0